# Flashaid temp

## Light Preprocessing

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import LearningRateScheduler
import joblib

In [2]:
flashaidtemp = pd.read_csv('flash360XY.csv')

In [3]:
flashaidtemp = flashaidtemp.drop(columns=['Total Count', 'Insurance Company', 'Proportional Claims Reported', 'Proportional Total Policies', 'Maternity Months', 'Room Rent Normal Propotinate Applicable', 'Room Rent ICU Propotinate Applicable'])

In [4]:
cols = ['Frequency Months', 'Adult', 'Child', 'Sum Assured',
       'Insurance Price with GST', 'Max Age', 'Min Age',
       'Insurance Type_Health', 'Insurance Type_Accidental Death', 'Insurance Type_Critical illness',
       'Pre-existing disease waiting period years',
       'Day Care Surgery waiting period years', 'Cooling Period days',
       'Room Rent ICU Available', 'Room Rent Normal Percent',
       'Room Rent ICU Percent', 'Cataract Available', 'Cataract waiting years',
       'Maternity Available', 'Maternity Flat', 'Hospital Covered',
       'Pre Hospitalization 30 days', 'Post Hospitalization 60 days',
       'Advanced Treatment Available', 'Accident Available',
       'Critical Available', 'HospiCash Available', 'Claim Risk']
flashaidtemp = flashaidtemp[cols]

In [5]:
synthetic_data = flashaidtemp.iloc[:156].copy() #implement only for health

# Set Maternity related columns to 0 or appropriate values
synthetic_data['Maternity Available'] = 0
synthetic_data['Maternity Flat'] = 0
synthetic_data['Insurance Price with GST'] = flashaidtemp['Insurance Price with GST'] * 0.75

# Append synthetic data to the original DataFrame
flashaidtemp = flashaidtemp.append(synthetic_data, ignore_index=True)

C:\Users\abhay\AppData\Local\Temp\ipykernel_26608\1989791682.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  flashaidtemp = flashaidtemp.append(synthetic_data, ignore_index=True)


In [6]:
#flash360temp.info()

In [7]:
flashaidtemp.to_csv('flashaidtemp.csv', index=False)

## If-Else

In [8]:
'''def get_user_input(df):
    user_input = {}
    for column in df.columns[:-1]:  # Exclude the 'Claim Ratio' column
        if column == 'Cataract waiting years':
            if user_input.get('Cataract Available') == 0:
                continue
        elif column == 'Maternity Flat':
            if user_input.get('Maternity Available') == 0:
                continue
        
        value = input(f"Enter {column}: ")
        
        # Convert input to appropriate type
        if df[column].dtype == 'int64':
            user_input[column] = int(value)
        elif df[column].dtype == 'float64':
            user_input[column] = float(value)
        else:
            user_input[column] = value

    return user_input'''

'def get_user_input(df):\n    user_input = {}\n    for column in df.columns[:-1]:  # Exclude the \'Claim Ratio\' column\n        if column == \'Cataract waiting years\':\n            if user_input.get(\'Cataract Available\') == 0:\n                continue\n        elif column == \'Maternity Flat\':\n            if user_input.get(\'Maternity Available\') == 0:\n                continue\n        \n        value = input(f"Enter {column}: ")\n        \n        # Convert input to appropriate type\n        if df[column].dtype == \'int64\':\n            user_input[column] = int(value)\n        elif df[column].dtype == \'float64\':\n            user_input[column] = float(value)\n        else:\n            user_input[column] = value\n\n    return user_input'

In [9]:
'''user_input = get_user_input(flash360temp)

user_input_df = pd.DataFrame([user_input])'''

'user_input = get_user_input(flash360temp)\n\nuser_input_df = pd.DataFrame([user_input])'

In [10]:
#user_input_df

In [11]:
'''def get_claim_ratio(df, user_input):
    df = flash360temp
    if user_input.get('Cataract Available') == 0:
        df = df.drop(columns=['Cataract waiting years'])
        user_input.pop('Cataract waiting years', None)
    if user_input.get('Maternity Available') == 0:
        df = df.drop(columns=['Maternity Flat'])
        user_input.pop('Maternity Flat', None)

    for index, row in df.iterrows():
        match = True
        for key, value in user_input.items():
            if row[key] != value:
                match = False
                break
        if match:
            return row['Claim Ratio']

    return "No matching record found."'''

'def get_claim_ratio(df, user_input):\n    df = flash360temp\n    if user_input.get(\'Cataract Available\') == 0:\n        df = df.drop(columns=[\'Cataract waiting years\'])\n        user_input.pop(\'Cataract waiting years\', None)\n    if user_input.get(\'Maternity Available\') == 0:\n        df = df.drop(columns=[\'Maternity Flat\'])\n        user_input.pop(\'Maternity Flat\', None)\n\n    for index, row in df.iterrows():\n        match = True\n        for key, value in user_input.items():\n            if row[key] != value:\n                match = False\n                break\n        if match:\n            return row[\'Claim Ratio\']\n\n    return "No matching record found."'

In [12]:
'''claim_ratio = -1
claim_ratio = get_claim_ratio(flash360temp, user_input)

if claim_ratio == -1:
    print("Check for wrong information entered.")
else:
    print(f"The Claim Ratio is: {claim_ratio}")'''

'claim_ratio = -1\nclaim_ratio = get_claim_ratio(flash360temp, user_input)\n\nif claim_ratio == -1:\n    print("Check for wrong information entered.")\nelse:\n    print(f"The Claim Ratio is: {claim_ratio}")'

## Random Forest

In [13]:
X = flashaidtemp.drop(columns=['Claim Risk'])
y = flashaidtemp['Claim Risk']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
rf_model = RandomForestRegressor(random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 6.653700044775924e-06


## ANN

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [31]:
def lr_schedule(epoch, lr):
    if epoch > 50:
        lr = lr * 0.5
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

In [41]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, callbacks=[lr_scheduler])

Epoch 1/100
14/14 [==============================] - 1s 8ms/step - loss: 0.4697 - val_loss: 0.0936 - lr: 0.0010
Epoch 2/100
14/14 [==============================] - 0s 3ms/step - loss: 0.2097 - val_loss: 0.0478 - lr: 0.0010
Epoch 3/100
14/14 [==============================] - 0s 3ms/step - loss: 0.1102 - val_loss: 0.0326 - lr: 0.0010
Epoch 4/100
14/14 [==============================] - 0s 3ms/step - loss: 0.0637 - val_loss: 0.0215 - lr: 0.0010
Epoch 5/100
14/14 [==============================] - 0s 3ms/step - loss: 0.0997 - val_loss: 0.0169 - lr: 0.0010
Epoch 6/100
14/14 [==============================] - 0s 2ms/step - loss: 0.0640 - val_loss: 0.0149 - lr: 0.0010
Epoch 7/100
14/14 [==============================] - 0s 2ms/step - loss: 0.0616 - val_loss: 0.0095 - lr: 0.0010
Epoch 8/100
14/14 [==============================] - 0s 3ms/step - loss: 0.0450 - val_loss: 0.0086 - lr: 0.0010
Epoch 9/100
14/14 [==============================] - 0s 2ms/step - loss: 0.0480 - val_loss: 0.0109 - lr:

In [42]:
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

3/3 [==============================] - 0s 0s/step - loss: 0.0022
Test Loss: 0.002186686499044299


## Save models

In [43]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [39]:
joblib.dump(rf_model, 'rf_model.pkl')

['rf_model.pkl']

In [44]:
model.save('ann_model.keras')